In [8]:
import pandas as pd
import pickle
import os

# Setup for pickling
dest = '..\\pickles'
if not os.path.exists(dest):
    os.makedirs(dest)


In [9]:
# import cleaned data
train_data = pd.read_csv('../data/cleaned_data.csv', index_col='id')

In [10]:
# Create and pickle things for text transformation (Stop words, stemmer, lematizer)

In [13]:
# Create and pickle hashing vectorizer with best params
from sklearn.feature_extraction.text import HashingVectorizer

hash_vectorizer = HashingVectorizer(alternate_sign=True, n_features=600000, ngram_range=(1,1))
vectorized_train_data = hash_vectorizer.fit_transform(train_data.comment_text)

pickle.dump(hash_vectorizer, open(os.path.join(dest, 'hash_vect.pkl'), 'wb'), protocol=4)

In [14]:
# Select columns and split data
from sklearn.model_selection import train_test_split
y = train_data.target
X = vectorized_train_data

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                    test_size=.2, random_state=1, stratify=y)

In [16]:
# Create, train, and pickle model
from sklearn.linear_model import LogisticRegression
best_lr = LogisticRegression(random_state=0, solver='saga', penalty='l2', C=1.0).fit(X_train, y_train)
pickle.dump(hash_vectorizer, open(os.path.join(dest, 'trained_log_reg.pkl'), 'wb'), protocol=4)